In [917]:
from cspython.merging_processing import combine_dfs
from cspython.match_dataset_creation import match_dataset_creation
import cPickle as pkl
from cspython.scraper import modifiedSoup
from cspython.data_processing import process_scrapped
import cPickle as pkl
import sys
sys.setrecursionlimit(15000)

import cspython.analysis as a
import scipy
print('scipy: {}'.format(scipy.__version__)) # numpy
import numpy as np
print('numpy: {}'.format(np.__version__))
# matplotlib


import matplotlib 
import matplotlib.pyplot as plt
print('matplotlib: {}'.format(matplotlib.__version__)) # pandas
import pandas as pd
print('pandas: {}'.format(pd.__version__))
# scikit-learn
#import sklearn
#print('sklearn: {}'.format(sklearn.__version__))
#import xlrd

import seaborn as sns
sns.set_style("whitegrid")
%matplotlib inline


import sklearn
print('sklearn: {}'.format(sklearn.__version__))
#import xlrd
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.linear_model import Lasso, Ridge  
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import VotingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.feature_selection import RFE
import xgboost as xgb
from sklearn import cross_validation
from sklearn.model_selection import GridSearchCV
from sklearn import cross_validation #might be model_selection <--- this is the new one
from sklearn.preprocessing import Normalizer
import seaborn as sns
from sklearn import preprocessing 
sns.set_style("whitegrid")
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
import pdb
from sklearn.ensemble import VotingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.feature_selection import RFE
from sklearn import cross_validation
from sklearn.model_selection import GridSearchCV
from sklearn import model_selection
from sklearn.feature_selection import RFECV

scipy: 0.19.1
numpy: 1.12.1
matplotlib: 2.0.2
pandas: 0.20.3
sklearn: 0.19.0


In [923]:
pd.set_option('display.max_columns', 300)

In [924]:
with open('../cspython/esl_teams.pkl', 'rb') as f: 
     d = pkl.load(f)

In [925]:
big_data = process_scrapped(d)
overview, big_data = big_data

In [926]:
data = combine_dfs(overview, big_data)

In [927]:
data = match_dataset_creation(data)

In [ ]:
data.loc[(data.loc[:,'player_team_name'] == 'Cloud9') & (data.loc[:,'map'] == 'Inferno'),].sort_values(by='date',ascending=True)

In [929]:
#data.to_pickle("data_for_analysis.pkl")


In [930]:
data = pd.read_pickle("data_for_analysis.pkl")

In [100]:
data_adv = data.loc[:, 'first_kills_sum_dr_hist':'player_team_opponent']

In [101]:
data_adv['date'] = data['date']

In [102]:
data_adv['match_id'] = data['match_id']

In [103]:
data_adv['player_team_name'] = data['player_team_name']

In [104]:
data_adv['map'] = data['map']

In [105]:
data_adv['winner_of_match']= data['winner_of_match']

In [106]:
data_adv = data_adv.drop(['team_A_score_Count','team_B_score_Count'], axis = 1)

In [107]:
data_adv = data_adv.groupby(['match_id', 'player_team_name', 'player_team_opponent','map', 'winner_of_match', 'date']).mean()

In [108]:
data_adv = pd.DataFrame(data_adv)
data_adv = data_adv.reset_index()

In [109]:
data_adv = data_adv.sort_values('date')

In [110]:
data_adv = data_adv.drop(['Mirage_win_his','Mirage_loss_his','Train_win_his','Train_loss_his','Cobblestone_win_his'
                      ,'Cobblestone_loss_his','Cache_win_his','Cache_loss_his','Inferno_win_his','Inferno_loss_his'
                       ,'Overpass_win_his','Overpass_loss_his','Nuke_win_his','Nuke_loss_his','who_kill_who_Cache_dr_hist'
                       ,'who_kill_who_Cobblestone_dr_hist','who_kill_who_Inferno_dr_hist','who_kill_who_Mirage_dr_hist'
                       ,'who_kill_who_Nuke_dr_hist','who_kill_who_Overpass_dr_hist','who_kill_who_Train_dr_hist'
                       , 'who_kill_who_sum_dr_hist'], axis = 1)#'player_team_name_CLG','player_team_name_Cloud9', 'player_team_name_Ghost'
#                       ,'player_team_name_Immortals', 'player_team_name_Liquid','player_team_name_Luminosity'
#                       , 'player_team_name_Misfits','player_team_name_NRG', 'player_team_name_OpTic','player_team_name_Renegades'
#                       , 'player_team_name_Rogue','player_team_name_SK', 'player_team_name_Splyce','player_team_name_compLexity','map_Cache'
#                       ,'map_Cobblestone','map_Inferno','map_Mirage','map_Nuke','map_Overpass','map_Train'
#                       ,'player_team_opponent_CLG','player_team_opponent_Cloud9','player_team_opponent_Ghost'
#                       ,'player_team_opponent_Immortals','player_team_opponent_Liquid','player_team_opponent_Luminosity'
#                       ,'player_team_opponent_Misfits','player_team_opponent_NRG','player_team_opponent_OpTic'
#                       ,'player_team_opponent_Renegades','player_team_opponent_Rogue','player_team_opponent_SK'
#                       ,'player_team_opponent_Splyce','player_team_opponent_compLexity'], axis = 1)

In [111]:
data_adv_even = data_adv.iloc[::2]  # even
data_adv_odd = data_adv.iloc[1::2]


In [112]:
data_adv_merged = pd.merge(data_adv_even, data_adv_odd, on =['match_id', 'date'])

In [113]:
pd.set_option('display.max_columns', 300)

In [114]:
data_adv.shape

(1208, 66)

In [115]:
data_adv_merged.loc[:,data_adv_merged.columns != 'date'] = data_adv_merged.loc[:,data_adv_merged.columns != 'date'].apply(pd.to_numeric, errors='ignore')

In [116]:
datay = data_adv_merged.loc[:,['winner_of_match_x', 'player_team_name_x', 'match_id']]

In [117]:
data_adv_merged = pd.get_dummies(data_adv_merged,columns = ['map_x'])

In [118]:
data_adv_merged.shape

(544, 136)

In [119]:
datay.loc[datay.winner_of_match_x != datay.player_team_name_x, 'winner_of_match_x'] = 0
datay.loc[datay.winner_of_match_x == datay.player_team_name_x, 'winner_of_match_x'] = 1


In [120]:
datay.winner_of_match_x = datay.winner_of_match_x.apply(pd.to_numeric, errors='ignore')

In [121]:
datay = datay.drop(['player_team_name_x','match_id'], axis = 1)

In [122]:
data_adv_merged = data_adv_merged.drop(['match_id'], axis = 1)

In [123]:
for a in data_adv_merged:
    print a

player_team_name_x
player_team_opponent_x
winner_of_match_x
date
first_kills_sum_dr_hist_x
awp_kills_sum_dr_hist_x
ADR_hist_x
Train_total_played_x
Cobblestone_total_played_x
Mirage_total_played_x
Cache_total_played_x
Inferno_total_played_x
Overpass_total_played_x
Nuke_total_played_x
Train_win_perc_map_x
Cobblestone_win_perc_map_x
Mirage_win_perc_map_x
Cache_win_perc_map_x
Inferno_win_perc_map_x
Overpass_win_perc_map_x
Nuke_win_perc_map_x
rd_total_his_Ghost_x
rd_total_his_OpTic_x
rd_total_his_Liquid_x
rd_total_his_Luminosity_x
rd_total_his_CLG_x
rd_total_his_NRG_x
rd_total_his_Cloud9_x
rd_total_his_Immortals_x
rd_total_his_SK_x
rd_total_his_compLexity_x
rd_total_his_Rogue_x
rd_total_his_Misfits_x
rd_total_his_Splyce_x
rd_total_his_Renegades_x
total_team_rd_Cache_x
total_team_rd_Cobblestone_x
total_team_rd_Inferno_x
total_team_rd_Mirage_x
total_team_rd_Overpass_x
total_team_rd_Train_x
total_team_rd_Nuke_x
ADR_his_Cache_x
ADR_his_Cobblestone_x
ADR_his_Inferno_x
ADR_his_Mirage_x
ADR_his_Nu

In [134]:
data_adv_merged = data_adv_merged.round(4)

In [135]:
#data_adv_temp = data_adv.loc[:,[,'Train_win_perc_map','Cobblestone_win_perc_map','Mirage_win_perc_map'
#                                ,'Cache_win_perc_map','Inferno_win_perc_map','Overpass_win_perc_map','Nuke_win_perc_map']]

In [144]:
X = data_adv_merged

In [145]:
X = X.select_dtypes(include = ['int64', 'float64', 'uint8', 'bool','datetime64[ns]']).iloc[:, 1:]

In [128]:
y = datay.values.astype(int)
X = X.values

In [129]:
def local_cv(model, params):                             #KFOLD WITH GRID SEARCH
    param_grid = params
    kfold = cross_validation.KFold(n=num_instances, n_folds=num_folds, random_state=seed)
    grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring=scoring, cv=kfold)
    grid_result = grid.fit(X, y)
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    for params, mean_score, scores in grid_result.grid_scores_:
        print("%f (%f) with: %r" % (scores.mean(), scores.std(), params))

In [130]:
num_folds = 3
num_instances = len(X) 
seed = 7
scoring = 'accuracy'


In [131]:
#X = data_adv.iloc[:,top_33_important_features]

In [132]:
models = []
models.append(('LR', LogisticRegression(random_state = seed)))
#models.append(('LASSO', Lasso())) 
#models.append(('Ridge', Ridge())) 
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('NB', GaussianNB()))
models.append(('KNeighborsClassifier', KNeighborsClassifier()))#ewights = 'distance' 
models.append(('XGBClassifier', xgb.XGBClassifier()))
models.append(('GradientBoostingClassifier', GradientBoostingClassifier(random_state = seed)))
models.append(('AdaBoostClassifier', AdaBoostClassifier(random_state = seed)))
models.append(('RandomForestClassifier', RandomForestClassifier(random_state = seed)))
models.append(('ExtraTreesClassifier', ExtraTreesClassifier(random_state = seed)))
models.append(('DecisionTreeClassifier', DecisionTreeClassifier(random_state = seed)))

# evaluate each model in turn
results = []
names = []
for name, model in models:
    kfold = model_selection.TimeSeriesSplit()
    cv_results = model_selection.cross_val_score(model, X, y, cv=kfold, scoring = scoring)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

LR: 0.598039 (0.019299)
LDA: 0.588235 (0.021646)
NB: 0.666667 (0.029615)
KNeighborsClassifier: 0.578431 (0.035178)
XGBClassifier: 0.629902 (0.039974)
GradientBoostingClassifier: 0.620098 (0.009171)
AdaBoostClassifier: 0.598039 (0.012498)
RandomForestClassifier: 0.575980 (0.024263)
ExtraTreesClassifier: 0.605392 (0.063062)
DecisionTreeClassifier: 0.580882 (0.036519)


In [35]:
estimator = xgb.XGBClassifier()
rfe = RFECV(estimator,cv = kfold)
fit = rfe.fit(X,y)
print("Num of feature: %d") % fit.n_features_
#print("Selected features: %s") % fit.support_
print("Feature Ranking: %s") % fit.ranking_

Num of feature: 33
Feature Ranking: [ 1  1  1  1  1  1 14  1  7  8 11 18  4  1  2  1  1 12  1  1  1 17  1 10  5
  6  1  1  1 15  1 13  3 19  1  1  1  1 20  1  1  1  1  1  1  1  1  1  9  1
  1 16]


In [36]:
top_33_important_features = [] 
for b in range(0,len(fit.ranking_)):
    if fit.ranking_[b] == 1:
        top_33_important_features.append(b)
        print b,data_adv.columns[b]

0 first_kills_sum_dr_hist
1 who_kill_who_sum_dr_hist
2 awp_kills_sum_dr_hist
3 ADR_hist
4 Train_win_his
5 Train_loss_his
7 Cobblestone_win_his
13 Cache_win_his
15 Cache_total_played
16 Inferno_win_his
18 Inferno_total_played
19 Overpass_win_his
20 Overpass_loss_his
22 Nuke_win_his
26 Cobblestone_win_perc_map
27 Mirage_win_perc_map
28 Cache_win_perc_map
30 Overpass_win_perc_map
34 rd_total_his_Liquid
35 rd_total_his_Luminosity
36 rd_total_his_CLG
37 rd_total_his_NRG
39 rd_total_his_Immortals
40 rd_total_his_SK
41 rd_total_his_compLexity
42 rd_total_his_Rogue
43 rd_total_his_Misfits
44 rd_total_his_Splyce
45 rd_total_his_Renegades
46 total_team_rd_Cache
47 total_team_rd_Cobblestone
49 total_team_rd_Mirage
50 total_team_rd_Overpass


In [1054]:
for a in range(0, len(models)):
    model1 = models[a]
    for b in range(a+1, len(models)):
        model2 = models[b]
        for c in range(b+1, len(models)):
            model3 = models[c]
            estimators = []
            estimators.append(model1)
            estimators.append(model2)
            estimators.append(model3)
            ensemble = VotingClassifier(estimators, voting='soft')
            results = model_selection.cross_val_score(ensemble, X, y, cv=kfold, scoring= scoring)
            if results.mean() > .69:
                print(model1[0], model2[0], model3[0],results.mean(), results.std())

KeyboardInterrupt: 